In [7]:
%glue_version 3.0
%number_of_workers 2
%worker_type G.2X
%idle_timeout 6000
%region ap-northeast-2
%iam_role arn:aws:iam::499514681453:role/AWSGlueServiceRoleJupyter

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Setting Glue version to: 3.0
Previous number of workers: 5
Setting new number of workers to: 2
Previous worker type: G.1X
Setting new worker type to: G.2X
Current idle_timeout is 2880 minutes.
idle_timeout has been set to 6000 minutes.
Previous region: None
Setting new region to: ap-northeast-2
Region is set to: ap-northeast-2
Current iam_role is None
iam_role has been set to arn:aws:iam::499514681453:role/AWSGlueServiceRoleJupyter.


https://docs.aws.amazon.com/ko_kr/glue/latest/dg/aws-glue-programming-etl-format-iceberg.html <br>
IceBerg 테이블을 생성하기 위해서는 GLUE JOB 실행 파라미터로 --datalake-formats = iceberg 값을 줘야한다. 
GLUE 콘솔에서 JOB을 등록할때 실행 파라미터 값을 설정하거나, CLI 커맨드를 이용하여 JOB 을 등록할 떄 값을 설정해야 한다.
이값을 설정하지 않으면 코드 실행시 glue_catalog 플러그인을 찾을 수 없다는 오류가 발생한다. 
이에 반해, 이 노트북에서 처럼 glue interactive 모드를 사용하는 경우는 아래와 같이 %%configure 매직을 활용하여 datalake format 이 iceberg 로 설정한다.

In [9]:
%%configure 
{ "datalake-formats":"iceberg" }                    

The following configurations have been updated: {'datalake-formats': 'iceberg'}


In [1]:
%%sql
show tables in `sample`

Authenticating with profile=default
glue_role_arn defined by user: arn:aws:iam::499514681453:role/AWSGlueServiceRoleJupyter
Trying to create a Glue session for the kernel.
Worker Type: G.2X
Number of Workers: 2
Session ID: 90091c52-8d34-4f92-8fae-81eaf1aadfd0
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.2
--enable-glue-datacatalog true
--datalake-formats iceberg
Waiting for session 90091c52-8d34-4f92-8fae-81eaf1aadfd0 to get into ready status...
Session 90091c52-8d34-4f92-8fae-81eaf1aadfd0 has been created.
+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|  sample|             tbl-csv|      false|
|  sample|       tbl-flight-p5|      false|
|  sample|     tbl-flight-part|      false|
|  sample|tbl-flight-part-r...|      false|
|  sample|         tbl-flight4|      false|
+--------+--------------------+-----------+


https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.SparkSession.html
glue spark 에서 iceberg 테이블을 사용하기 위해서는 아래와 같이 config 를 설정해야 한다. 코드상에 spark 세션의 .config 함수를 활용할 수도 있고,
glue job 의 파리미터로 설정할 수도 있다. 지금처럼 인터랙티브 모드로 개발하는 경우에는 spark session 에 설정한다.

In [2]:
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from pyspark.sql import SparkSession

"""
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
SparkSession = glueContext.spark_session
job = Job(glueContext)
"""

# Iceberg configuration
warehouse_path = 's3://glue-seoul-20230405/iceberg'
spark = SparkSession.builder \
    .config('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions') \
    .config('spark.sql.catalog.glue_catalog', 'org.apache.iceberg.spark.SparkCatalog') \
    .config('spark.sql.catalog.glue_catalog.warehouse', warehouse_path) \
    .config('spark.sql.catalog.glue_catalog.catalog-impl', 'org.apache.iceberg.aws.glue.GlueCatalog') \
    .config('spark.sql.catalog.glue_catalog.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO') \
    .getOrCreate()

sc = spark.sparkContext
glueContext = GlueContext(sc)
#job = Job(glueContext)
sc

<SparkContext master=jes appName=GlueReplApp>


In [3]:
%%sql
select year, quarter, month from sample.`tbl-flight4` limit 3

+----+-------+-----+
|year|quarter|month|
+----+-------+-----+
|2016|      4|   10|
|2016|      4|   10|
|2016|      4|   10|
+----+-------+-----+


In [4]:
query = """
create table `glue_catalog`.sample.`iceberg`
using iceberg
select * from sample.`tbl-flight4`
"""
spark.sql(query)

DataFrame[]


In [5]:
%%sql
show tables in `sample`

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|  sample|             iceberg|      false|
|  sample|             tbl-csv|      false|
|  sample|       tbl-flight-p5|      false|
|  sample|     tbl-flight-part|      false|
|  sample|tbl-flight-part-r...|      false|
|  sample|         tbl-flight4|      false|
+--------+--------------------+-----------+


In [8]:
%%sql
describe table `glue_catalog`.sample.`iceberg`

+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|           year|   bigint|       |
|        quarter|   bigint|       |
|          month|   bigint|       |
|   day_of_month|   bigint|       |
|    day_of_week|   bigint|       |
|        fl_date|   string|       |
| unique_carrier|   string|       |
|     airline_id|   bigint|       |
|        carrier|   string|       |
|       tail_num|   string|       |
|         fl_num|   bigint|       |
|   crs_dep_time|   bigint|       |
|       dep_time|   bigint|       |
|      dep_delay|   bigint|       |
|  dep_delay_new|   bigint|       |
|      dep_del15|   bigint|       |
|dep_delay_group|   bigint|       |
|   dep_time_blk|   string|       |
|       taxi_out|   bigint|       |
|     wheels_off|   bigint|       |
+---------------+---------+-------+
only showing top 20 rows


In [9]:
%%sql
select count(1) from `glue_catalog`.sample.`iceberg`

+--------+
|count(1)|
+--------+
| 5248439|
+--------+
